# 16 set 2018

O objetivo de hoje é:

- Obter todas as fotos de todos os candidatos que se cadastraram no TSE
- Disponibilizar todas as fotos em escala de cinza

In [1]:
import pandas as pd
import numpy as np
import requests
import sys
import os
import glob
import time

In [2]:
from utils.dataframe_to_database import dataframe_to_database
from utils.dataframe_to_file import dataframe_to_file
from utils.database_to_dataframe import database_to_dataframe
from utils.download import download

## Todos os registros cadastrados na hot table de candidatos

In [3]:
df_candidatos_2018 = database_to_dataframe(
    tabela='candidatura',
    colunas_tabela=['candidato_id', 'sequencial_candidato', 'sigla'],
    condicao='LEFT JOIN estado e on candidatura.estado_id = e.id WHERE (eleicao_id = 202 OR eleicao_id = 203) AND (candidatura.situacao_candidatura_id IS NULL OR candidatura.situacao_candidatura_id != 3)'
)

df_candidatos_2018['sigla'] = df_candidatos_2018.sigla.fillna('BR').astype(str)

df_candidatos_2018

,candidato_id,sequencial_candidato,sigla
0,1261292,10000602842,AC
1,957409,10000600143,AC
2,957477,10000601043,AC
3,1261158,10000610684,AC
4,1261175,10000607159,AC
5,1261233,10000601030,AC
6,969552,10000600804,AC
7,401312,10000600791,AC
8,1261222,10000600117,AC
9,968434,10000610660,AC


In [5]:
dataframe_to_file(
    dataframe=df_candidatos_2018,
    origem_dados='tse',
    nome='candidatos-id-sequencial-estado'
)

In [ ]:
falhas = []
total = len(df_candidatos_2018)
df_fotos = pd.DataFrame()

for candidato in df_candidatos_2018.itertuples():
    sys.stdout.write('\rCandidato {} de {}'.format(candidato.Index, total))
    sys.stdout.flush()

    while True:
        try:
            link = 'http://divulgacandcontas.tse.jus.br/divulga/rest/v1/candidatura/buscar/2018/{}/2022802018/candidato/{}'.format(
                candidato.sigla,
                candidato.sequencial_candidato
            )
            request = requests.get(link)
            response = request.json()

            df_i = pd.DataFrame(data = {
                'candidato_id': [candidato.candidato_id],
                'foto': [response['fotoUrl']]
            })

            if df_i.empty:
                raise Exception(candidato.candidato_id)
            else:
                df_fotos = df_fotos.append(df_i)
                break
        except:
            falhas.append(candidato.candidato_id)

Candidato 0 de 27361